In [1]:
# this sets up basic packages
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.cosmology.units as cu
import h5py

# this sets up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# this sets up astropy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky, Distance
from astropy.table import Table, unique, setdiff

from regions import Regions, CircleSkyRegion

In [2]:
dream = fits.open('/Users/ngbinh/Downloads/DREaM_main.fits')
dream_data = Table(dream[1].data)

In [3]:
# this selects all the neighbors with 2 < z < 5, then gets their Mstar and SFR
relevant = dream_data['ID', 'RA', 'Dec', 'redshift', 'M_gal', 'logpsi'][dream_data['redshift']<5]
relevant = relevant[relevant['redshift']>2].to_pandas()

In [4]:
# this selects all the ones that are quenched (i.e. sSFR < -9.8)
quenched_ones = relevant[np.log10(10**relevant['logpsi'] / relevant['M_gal']) < -9.8]
quenched_ones = quenched_ones[quenched_ones['M_gal'] > 10**10.5]

In [5]:
relevant

,ID,RA,Dec,redshift,M_gal,logpsi
0,13241306,0.489857,0.010564,2.014664,1.273912e+07,-1.369086
1,13241307,0.489621,0.015140,2.014622,1.961021e+05,-2.058861
2,13241308,0.490752,-0.043852,2.014522,7.534733e+05,-2.632520
3,13241309,0.490896,-0.043992,2.014489,2.077482e+05,1.622524
4,13241310,0.492010,-0.043814,2.014393,1.295432e+05,-3.098835
...,...,...,...,...,...,...
12343026,25951780,-0.395015,0.470712,4.977378,1.483325e+05,1.476693
12343027,25951781,-0.325127,0.410665,4.984012,3.898026e+05,-1.798956
12343028,25951782,-0.052119,-0.499995,4.966891,2.134280e+05,-2.114437
12343029,25951783,0.170450,-0.499948,4.950807,2.765631e+06,-1.781149


There are too many quenched galaxies in this catalog, so let's just randomly sample 100 of them.

In [7]:
# this selects all the ones that are NOT quenched in relevant. the potential neighbors will be in here.
others = pd.concat([relevant, quenched_ones]).drop_duplicates(keep=False)

In [8]:
quenched_sample = quenched_ones.sample(n=100)

In [9]:
# this creates SkyCoord objects for the quenched sample and the non-quenched ones
quenched_cat = SkyCoord(ra=quenched_sample['RA'].values*u.degree, dec=quenched_sample['Dec'].values*u.degree)
others_cat = SkyCoord(ra=others['RA'].values*u.degree, dec=others['Dec'].values*u.degree)

In [37]:
all_neighbor_ids = []

# this looks for neighbors for each of the quenched ones within 15 arcsec
for quenched_object in quenched_cat:
    
    # use separation to find nearby objects ("nearby" in terms of RA and Dec) in the catalog
    d2d = quenched_object.separation(others_cat)
    catalogmsk = d2d < 15*u.arcsec
    neighbor_ids = others['ID'].values[catalogmsk]

    all_neighbor_ids.append(neighbor_ids)

In [45]:
all_neighbor_ids = np.array(all_neighbor_ids, dtype='object')

In [51]:
quenched_sample

,ID,RA,Dec,redshift,M_gal,logpsi
4667165,18055008,-0.214036,-0.460370,2.912464,3.938851e+10,-0.514176
3610210,16998053,0.025924,0.188798,2.627664,3.459804e+10,-0.562408
1052767,14440610,-0.229286,0.292428,2.182852,6.731970e+10,-0.168675
3297095,16684938,0.112391,-0.373233,2.534276,4.500272e+10,0.078917
5108049,18495892,-0.324630,0.243035,2.861275,6.588794e+10,0.225653
...,...,...,...,...,...,...
5527793,18915636,-0.166495,-0.253958,2.968608,3.844374e+10,-0.407250
493882,13881725,-0.011477,0.197965,2.080440,1.081174e+11,0.046948
1457005,14844848,0.025664,0.412885,2.232555,1.112780e+11,0.129484
3426810,16814653,0.376584,-0.412840,2.572012,3.750012e+10,-0.691502


# CRITERION 1: |z$_{QG}$ - z$_{gal}$| < 0.5

In [79]:
for idx in range(100):
    # this retriesves z_QG
    z_QG = quenched_sample['redshift'].values[idx]

    # this retrieves the IDs of the neighbor candidates around the QG
    neighbor_id_array = all_neighbor_ids[idx]

    # this locates the galaxies with the IDs above, then retrieves their photometric redshifts and IDs.
    # note: the IDs are retrieved in order to keep track of which galaxies we're talking about.
    neighbor_zphot_df = others.loc[others['ID'].isin(neighbor_id_array)][['ID', 'redshift']]
    
    # this sets the criterion
    criterion = (np.abs(z_QG - neighbor_zphot_df['redshift'].values) <= 0.5) 

    # this saves the IDs of the neighbors that pass criterion 1
    passes = neighbor_zphot_df[criterion]

    print(passes)

               ID  redshift
2355489  15743332  2.471031
2355965  15743808  2.472248
2362207  15750050  2.478723
2362208  15750051  2.478713
2362209  15750052  2.478702
...           ...       ...
7165675  20553518  3.340922
7165747  20553590  3.344575
7216443  20604286  3.352785
7302347  20690190  3.369459
7303496  20691339  3.374633

[349 rows x 2 columns]
               ID  redshift
633105   14020948  2.165452
633106   14020949  2.165444
633107   14020950  2.165486
633109   14020952  2.165062
633110   14020953  2.165049
...           ...       ...
5829850  19217693  3.027429
5841785  19229628  3.011168
5841821  19229664  3.010645
5841822  19229665  3.010503
5842079  19229922  3.008270

[361 rows x 2 columns]
               ID  redshift
88851    13330157  2.013467
200658   13528234  2.035219
200718   13528294  2.034420
200830   13528406  2.035976
205918   13533494  2.043658
...           ...       ...
3996300  17384143  2.633857
3996301  17384144  2.633875
4315203  17703046  2.680582
